In [5]:
!pip3 install genpipes
!pip3 install nltk

Looking in indexes: https://dtn.jfrog.io/artifactory/api/pypi/pypi/simple
  Using cached https://dtn.jfrog.io/artifactory/api/pypi/pypi/packages/packages/a6/25/da9dc4f8ff0e38e75edbbef43aff2e64c972746778890883f83543c441ba/genpipes-1.0.0-py3-none-any.whl (4.8 kB)

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python3.10 -m pip install --upgrade pip
Looking in indexes: https://dtn.jfrog.io/artifactory/api/pypi/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 39.5 kB/s eta 0:00:0000:0100:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 34.1 kB/s eta 0:00:00a 0:00:01
  Using cached https://dtn.jfrog.io/artifactory/api/pypi/pypi/packages/packages/47/bb/849011636c4da2e44f1253cd927cfb20ada4374d8b3a4e425416e84900cc/tqdm-4.64.1-py2.py3-none-any.whl (78 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.9/293.9 kB 32.5 kB/s eta 0:00:00a 0:00:01

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run:

In [15]:
import string

import xml.etree.ElementTree as ET
import pandas as pd
from nltk.tokenize import word_tokenize, sent_tokenize  
import nltk
import os

In [16]:
tags = ["I-NAME", "I-PROFESSION", "I-LOCATION", "I-AGE", "I-DATE", "I-CONTACT", "I-ID", "I-PHI",  "O"]
nltk.download('punkt')

class Tags():
    def __init__(self):
        self.entities = []
        self.tags = []
        self.sub_tags = [] 
    
    def append_entity(self, entity: list):
        self.entities.extend(entity)
    
    def append_tag(self, tag: list):
        self.tags.extend(tag)
    
    def append_sub_tags(self, sub_tag: list):
        self.sub_tags.extend(sub_tag)
    

def gather_tags(root):
    entities = []
    tags = []
    sub_tags = []
    for child in root.iter('TAGS'):
        for subchild in child:
            entity = subchild.get("text").split(" ")
            [tags.append(subchild.tag) for ent in entity]
            [sub_tags.append(subchild.get("TYPE")) for ent in entity]
            entities.extend(entity)
    
    tags.append("O")
    return entities, tags, sub_tags

def split_to_sent(root):
    doc_txt = str(root.find('TEXT').text)
    tokens = sent_tokenize(doc_txt)

    return tokens

def split_to_tokens(text):
    tokens = word_tokenize(text, preserve_line=True)

    return tokens

def label_txt(root):
    new_tags = ["I-NAME", "I-PROFESSION", "I-LOCATION", "I-AGE", "I-DATE", "I-CONTACT", "I-ID", "I-PHI",  "O"]
    orig_tags = ["NAME", "PROFESSION", "LOCATION", "AGE", "DATE", "CONTACT", "ID", "PHI", "O"]
    entities, tags, sub_tags = gather_tags(root)
    
    sentences = split_to_sent(root)
    ents, toks = [], []
    
    for sentence in sentences:
        tokens = split_to_tokens(sentence)
        for token in tokens:
            idx = orig_tags.index(tags[entities.index(token) if token in entities else -1])
            ents.append(token) 
            toks.append(new_tags[idx])

        ents.append("")
        toks.append("")
    
    return ents, toks

     

        



[nltk_data] Downloading package punkt to
[nltk_data]     /Users/jayragaileortiz/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [17]:
listOfFile = os.listdir("./data/NEW-PHI/")
listOfFileDev = listOfFile[:len(listOfFile)//2]
listOfFileTest = listOfFile[len(listOfFile)//2:]
"""
ner_tags_dev = Tags()
for file in listOfFileDev:
    tree = ET.parse(f"./data/test/{file}")
    root = tree.getroot()
    label_txt(root, "dev.txt")
"""
    

'\nner_tags_dev = Tags()\nfor file in listOfFileDev:\n    tree = ET.parse(f"./data/test/{file}")\n    root = tree.getroot()\n    label_txt(root, "dev.txt")\n'

In [18]:
"""ner_tags = Tags()
for file in listOfFileTest:
    tree = ET.parse(f"./data/test/{file}")
    root = tree.getroot()
    label_txt(root, "test.txt")"""

listOfFile = os.listdir("./data/NEW-PHI/")
listOfFileDev = listOfFile[:len(listOfFile)//2]
listOfFileTest = listOfFile[len(listOfFile)//2:]

entities, tokens = [], []
for file in listOfFileTest:
    tree = ET.parse(f"./data/NEW-PHI/{file}")
    root = tree.getroot()
    ents, toks = label_txt(root)
    entities.extend(ents)
    tokens.extend(toks) 

new_tokens = [entities, tokens]
df = pd.DataFrame(new_tokens).transpose()
print(df)
    

               0  1
0         Record  O
1           date  O
2              :  O
3     2020-05-30  O
4             CC  O
...          ... ..
2156         and  O
2157  reconciled  O
2158        with  O
2159     patient  O
2160               

[2161 rows x 2 columns]


In [19]:
df.to_csv("new-test.txt", sep=" ", index=False, columns=None)